<a href="https://colab.research.google.com/github/AnuragSharma5893/Dashboard_Visualizer-ML-Project/blob/main/Dashboard_Visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%pip install comet_ml torch datasets transformers scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached comet_ml-3.31.15-py3-none-any.whl (412 kB)
  Using cached datasets-2.5.2-py3-none-any.whl (432 kB)
     |████████████████████████████████| 4.9 MB 5.0 MB/s 
     |████████████████████████████████| 495 kB 43.1 MB/s 
     |████████████████████████████████| 162 kB 39.0 MB/s 
     |████████████████████████████████| 54 kB 3.0 MB/s 
     |████████████████████████████████| 54 kB 2.9 MB/s 
     |████████████████████████████████| 140 kB 49.7 MB/s 
     |████████████████████████████████| 127 kB 67.3 MB/s 
     |████████████████████████████████| 162 kB 72.0 MB/s 
     |████████████████████████████████| 158 kB 79.3 MB/s 
     |████████████████████████████████| 157 kB 77.1 MB/s 
     |████████████████████████████████| 157 kB 58.1 MB/s 
     |████████████████████████████████| 157 kB 65.0 MB/s 
     |████████████████████████████████| 157 kB 69.5 MB/s 
     |████████████████████████████████

Initailize Comet


In [5]:
import comet_ml

comet_ml.init(project_name = "imdb-distrilbart")

Please enter your Comet API key from https://www.comet.com/api/my/settings/
(api key may not show as you type)
Comet API key: ··········


COMET INFO: Comet API key is valid
COMET WARNING: running in Google Colab, but can't find mounted drive. Using HOME instead
COMET WARNING: if drive is mounted, set COMET_CONFIG to save config there
COMET INFO: Comet API key saved in /root/.comet.config


Set Model Type

In [6]:
PRE_TRAINED_MODEL_NAME = "distilbert-base-uncased"
SEED = 20

Load Data

In [10]:
!pip install transformers 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
from transformers import AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset

raw_datasets = load_dataset("imdb")

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Setup Tokenizer

In [13]:
tokenizer = AutoTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Define the we are going to defined the tokenize function and we are going to be created a brand new tokenized datasets where we pass our raw dataset in order to get a tokenized version of those dataset.

In [14]:
def tokenize_function(examples):
  return tokenizer(examples["text"],padding="max_length", truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)  

  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/50 [00:00<?, ?ba/s]

In [16]:
!pip install  transformers 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [18]:
from transformers import DataCollatorWithPadding 

data_collator= DataCollatorWithPadding(tokenizer=tokenizer)

Create Sample Datasets

For this, we are only going to sample 200 examples from our dataset.

In [22]:
train_dataset = tokenized_datasets["train"].shuffle(SEED).select(range(200))
eval_dataset = tokenized_datasets["test"].shuffle(SEED).select(range(200))

Setup Transformer Model

In [23]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    PRE_TRAINED_MODEL_NAME, num_labels=2
)

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier

Setup Evaluation function 

In [26]:
from six import iteritems
from sklearn.metrics import accuracy_score , precision_recall_fscore_support 

def get_example(index):
  return eval_dataset[index]["text"]

def compute_metrics(pred):
  experiment = comet_ml.get_global_experiment()
  
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  precision,recall, f1, _ = precision_recall_fscore_support(
      labels, preds, average="macro"
  )
  acc = accuracy_score(labels,preds)

  if experiment:
    epoch = int(experiment.curr_epoch) if experiment.curr_epoch is not None else 0
    experiment.set_epoch(epoch)
    experiment.log_confusion_matrix(
        y_true = labels,
        y_predicted = preds,
        file_name = f"confusion-matrix-epoch-{epoch}.json",
        labels = ["negative","positive"],
        index_to_example_function = get_example
    )
    for i in range(20):
      experiment.log_text(get_example(i), metadata={"label": labels[i].item()})

    return {"accuracy":acc, "f1": f1, "precision": precision,"recall": recall}  


Run Training 

In [35]:
%env COMET_MODE=ONLINE
%env COMET_LOG_ASSETS=TRUE

training_args = TrainingArguments(
    seed=SEED,
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=1,
    do_train=True,
    do_eval=True,
    evaluation_strategy="steps",
    eval_steps=25,
    save_strategy="steps",
    save_total_limit=10,
    save_steps=25,
    per_device_train_batch_size=8
)

trainer = Trainer (
    model=model,
    args= training_args,
    train_dataset= train_dataset,
    eval_dataset= eval_dataset,
    compute_metrics= compute_metrics,
    data_collator=data_collator
)
trainer.train()

env: COMET_MODE=ONLINE
env: COMET_LOG_ASSETS=TRUE


The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 200
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 25
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET

Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
25,No log,0.667147,0.670000,0.663403,0.693218,0.674807


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 200
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-25
Configuration saved in ./results/checkpoint-25/config.json
Model weights saved in ./results/checkpoint-25/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Logging checkpoints. This may take time.
COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/anuragsharma5893/imdb-distrilbart/ecb2ca1898ad44d3ba6d70ca21965d8d
COMET INFO:   Metrics [count] (min, max):
COMET 

TrainOutput(global_step=25, training_loss=0.6914327239990234, metrics={'train_runtime': 728.0067, 'train_samples_per_second': 0.275, 'train_steps_per_second': 0.034, 'total_flos': 26493479731200.0, 'train_loss': 0.6914327239990234, 'epoch': 1.0})